# Covid-19 Data Science Project

Eine Datenanalyse zu Covid-19 Daten. Ziel dieser Datenanalyse ist es ein Modell zu entwickeln, dass die zukünftigen Fallzahlen anhand verschiedener Input-Variablen wie dem Standort, den aktuellen Maßnahmen (Lockdown) und den vorherigen Fallzahlen vorhersagen kann. Zur Analyse werden Regressionsmodelle verwendet. Anschließend wird ein Deep Learning Algorithmus (neuronales Netz) angewendet.

In [93]:
import io
import urllib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from progressbar import ProgressBar

Historische Daten vom RKI zum Bundesland Baden-Württemberg.

In [4]:
url_cases_rki = "https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv?where=IdBundesland%20%3E%3D%208%20AND%20IdBundesland%20%3C%3D%208"
filename_cases = "./RKI_daily.csv"

urllib.request.urlretrieve(url_cases_rki, filename_cases)

('./RKI_daily.csv', <http.client.HTTPMessage at 0x20726688f48>)

In [6]:
data_all = pd.read_csv("RKI_daily.csv")

<br>
<span style="color:red">
    Nächste Schritte für die Daten:
</span>

- Aufsummieren von AnzahlFall nach Meldedatum und Landkreis
- Aufsummieren von AnzahlFall nach Meldedatum, Landkreis und für Altergruppe 65+ (Vermutung... benötigen einfach Fälle bei älteren Personen)
- Droppen von irrelevanten Spalten
- Neue Spalte für die 7-Tage-Inzidenz (vielleicht mittels Sortierung nach Meldedatum und Landkreis? Könnten dann aber Tage fehlen)
- Neue Spalte für Anzahl Fälle alte Personen letzte 7 Tage
- Löschen aller Zeilen außer Freitag (mein Vorschlag) -> Dann Vorhersagen von Freitag zu Freitag

Berechnung der 7-Tage-Inzidenz mittels Meldedatum: https://lua.rlp.de/de/presse/detail/news/News/detail/corona-hinweise-zur-berechnung-der-7-tage-inzidenz/

In [7]:
data_all

,ObjectId,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
0,482501,8,Baden-Württemberg,SK Stuttgart,A05-A14,M,1,0,2020/05/22 00:00:00,8111,"25.01.2021, 00:00 Uhr",0,-9,2020/05/12 00:00:00,0,1,1,Nicht übermittelt
1,482502,8,Baden-Württemberg,SK Stuttgart,A05-A14,M,1,0,2020/06/08 00:00:00,8111,"25.01.2021, 00:00 Uhr",0,-9,2020/06/08 00:00:00,0,1,0,Nicht übermittelt
2,482503,8,Baden-Württemberg,SK Stuttgart,A05-A14,M,1,0,2020/06/12 00:00:00,8111,"25.01.2021, 00:00 Uhr",0,-9,2020/06/09 00:00:00,0,1,1,Nicht übermittelt
3,482504,8,Baden-Württemberg,SK Stuttgart,A05-A14,M,1,0,2020/06/18 00:00:00,8111,"25.01.2021, 00:00 Uhr",0,-9,2020/06/17 00:00:00,0,1,1,Nicht übermittelt
4,482505,8,Baden-Württemberg,SK Stuttgart,A05-A14,M,1,0,2020/06/19 00:00:00,8111,"25.01.2021, 00:00 Uhr",0,-9,2020/06/19 00:00:00,0,1,0,Nicht übermittelt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156200,642477,8,Baden-Württemberg,LK Sigmaringen,unbekannt,M,1,0,2020/10/30 00:00:00,8437,"25.01.2021, 00:00 Uhr",0,-9,2020/10/30 00:00:00,0,1,0,Nicht übermittelt
156201,642478,8,Baden-Württemberg,LK Sigmaringen,unbekannt,W,1,0,2020/10/26 00:00:00,8437,"25.01.2021, 00:00 Uhr",0,-9,2020/10/26 00:00:00,0,1,0,Nicht übermittelt
156202,642479,8,Baden-Württemberg,LK Sigmaringen,unbekannt,W,1,0,2020/10/28 00:00:00,8437,"25.01.2021, 00:00 Uhr",0,-9,2020/10/26 00:00:00,0,1,1,Nicht übermittelt
156203,642480,8,Baden-Württemberg,LK Sigmaringen,unbekannt,W,1,0,2020/11/01 00:00:00,8437,"25.01.2021, 00:00 Uhr",0,-9,2020/10/26 00:00:00,0,1,1,Nicht übermittelt


<br>
Bevölkerungsentwicklung der Stadt- und Landkreise in Baden-Württemberg
https://www.statistik-bw.de/BevoelkGebiet/Bevoelkerung/01035055.tab?R=LA

Bevölkerungsdichte der Stadt- und Landkreise in Baden-Württemberg
https://www.statistik-bw.de/BevoelkGebiet/Bevoelkerung/01515020.tab?R=LA

In [10]:
# url_inhab = "https://www.statistik-bw.de/BevoelkGebiet/Bevoelk_I_D_A_vj.csv"
# url_inhab_perkm2 = "https://www.statistik-bw.de/BevoelkGebiet/Bevoelk_I_Flaeche_j.csv"

# filename_inhab = "./BW_inhab.csv"
# filename_inhab_perkm2 = "./BW_inhab_perkm2.csv"

# urllib.request.urlretrieve(url_inhab, filename_inhab)
# urllib.request.urlretrieve(url_inhab_perkm2, filename_inhab_perkm2)

In [43]:
data_inhab = pd.read_csv("https://www.statistik-bw.de/BevoelkGebiet/Bevoelk_I_D_A_vj.csv",
                 encoding = "ISO-8859-1",
                 sep=";",
                 decimal=",",
                 skiprows=17)
data_inhab = data_inhab[(data_inhab["Amtlicher Gemeindeschlüssel (AGS)"]>1000) &
                        (data_inhab["Amtlicher Gemeindeschlüssel (AGS)"]<10000) &
                        (data_inhab["Bevölkerung insgesamt"].str.isnumeric())==True]

In [126]:
data_inhab

,Kürzel der Regionaleinheit,Amtlicher Gemeindeschlüssel (AGS),Regionalname,Stichtag,Bevölkerung insgesamt,Bevölkerung männlich,Bevölkerung weiblich,Deutsche zusammen,Deutsche männlich,Deutsche weiblich,Ausländer zusammen,Ausländer männlich,Ausländer weiblich
17,KR,8111,Stadtkreis Stuttgart,31.03.2019,635443,317623,317820,477039,234397,242642,158404,83226,75178
18,KR,8115,Landkreis Böblingen,31.03.2019,392500,195510,196990,319810,157420,162390,72690,38090,34600
19,KR,8116,Landkreis Esslingen,31.03.2019,534127,267039,267088,440860,215729,225131,93267,51310,41957
20,KR,8117,Landkreis Göppingen,31.03.2019,257596,128314,129282,214986,105457,109529,42610,22857,19753
21,KR,8118,Landkreis Ludwigsburg,31.03.2019,544685,270549,274136,446860,218351,228509,97825,52198,45627
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7028,KR,8425,Landkreis Alb-Donau-Kreis,30.09.2020,198275,100047,98228,170568,84837,85731,27707,15210,12497
7029,KR,8426,Landkreis Biberach,30.09.2020,202322,101918,100404,179640,89404,90236,22682,12514,10168
7030,KR,8435,Landkreis Bodenseekreis,30.09.2020,218641,107779,110862,187656,91897,95759,30985,15882,15103
7031,KR,8436,Landkreis Ravensburg,30.09.2020,286240,142728,143512,252799,124070,128729,33441,18658,14783


In [133]:
data_inhab_perkm = pd.read_csv("https://www.statistik-bw.de/BevoelkGebiet/Bevoelk_I_Flaeche_j.csv",
                 encoding = "ISO-8859-1",
                 sep=";",
                 decimal=",",
                 skiprows=18)

In [135]:
data_inhab_perkm = data_inhab_perkm[(data_inhab_perkm2["Amtlicher Gemeindeschlüssel (AGS)"]>1000) &
                        (data_inhab_perkm["Amtlicher Gemeindeschlüssel (AGS)"]<10000)]

C:\Users\starkt2\AppData\Local\Continuum\anaconda3\envs\covidDL\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [136]:
data_inhab_perkm

,Kürzel der Regionaleinheit,Amtlicher Gemeindeschlüssel (AGS),Postleitzahl,Regionalname,Stichtag,Bevölkerung insgesamt,Gemeindegebiet ha,Bevölkerungsdichte EW/km²
17,KR,8111,X,Stadtkreis Stuttgart,31.12.2015,623738,20735,3008
18,KR,8115,X,Landkreis Böblingen,31.12.2015,381281,61782,617
19,KR,8116,X,Landkreis Esslingen,31.12.2015,524127,64148,817
20,KR,8117,X,Landkreis Göppingen,31.12.2015,252749,64236,393
21,KR,8118,X,Landkreis Ludwigsburg,31.12.2015,534074,68683,778
...,...,...,...,...,...,...,...,...
4704,KR,8425,X,Landkreis Alb-Donau-Kreis,31.12.2019,197076,135854,145
4705,KR,8426,X,Landkreis Biberach,31.12.2019,201282,140952,143
4706,KR,8435,X,Landkreis Bodenseekreis,31.12.2019,217470,66477,327
4707,KR,8436,X,Landkreis Ravensburg,31.12.2019,285424,163208,175


<br>
Altersstruktur der Stadt- und Landkreise in Ba-Wü
https://www.statistik-bw.de/BevoelkGebiet/Alter/98015200.tab?R=KR237

Download jeweils mittels des Amtlichen Gemeindeschlüssels (AGS)

In [122]:
pbar = ProgressBar()

AGS = ( pd.unique(data_inhab_perkm["Amtlicher Gemeindeschlüssel (AGS)"]) )%1000
data_age = pd.DataFrame(columns = ["Jahr", "Unter20", "Über65", "20bis65", "AGS"])

for ags in pbar(AGS):
    temp_data = pd.read_csv("https://www.statistik-bw.de/BevoelkGebiet/Alter/98015200.tab?R=KR" + str(ags) + "&form=csv",
                 encoding = "ISO-8859-1",
                 sep=";",
                 decimal=",",
                 names = ["Jahr", "Unter20", "Über65", "20bis65"],
                 usecols=[0,1,2,3],
                 skiprows = 28,
                 skipfooter = 20,
                    engine='python')
    temp_data["AGS"] = ags
    data_age = data_age.append(temp_data, ignore_index=True)

data_age["AGS"] = data_age["AGS"] + 8000

100% |########################################################################|


In [137]:
data_age

,Jahr,Unter20,Über65,20bis65,AGS
0,2020,109.680,113.716,421.919,8111
1,2021,109.892,114.218,423.461,8111
2,2020,78.538,80.109,236.644,8115
3,2021,78.881,81.539,236.339,8115
4,2020,101.883,111.712,325.572,8116
...,...,...,...,...,...
83,2021,40.311,50.212,126.459,8435
84,2020,56.464,58.085,172.557,8436
85,2021,56.517,59.122,172.422,8436
86,2020,25.482,27.600,78.546,8437


Weitere Datenquellen:
- a
- b